In [0]:
%run "/Workspace/Project/04.Common Notebook"

In [0]:
from pyspark.sql.functions import *

In [0]:
dbutils.widgets.text("Enter the environment","")
env = dbutils.widgets.get("Enter the environment")

In [0]:
def read_silver_traffic_data(environment):
    # Read traffic data from silver layer
    print("Reading silver traffic data:", end=' ')

    df_silver_traffic = spark.readStream.table(f"{environment}_catalog.silver.silver_traffic")
    
    print('Success !!')
    print("****************************************************************************************")
    return df_silver_traffic


In [0]:
def read_silver_roads_data(environment):
    # Read road data from silver layer
    print("Reading silver roads data:", end=' ')

    df_silver_roads = spark.readStream.table(f"{environment}_catalog.silver.silver_roads")
    
    print('Success !!')
    print("****************************************************************************************")
    return df_silver_roads


In [0]:
def create_vehicle_intensity(df):
    # Create VEHICLE_INTENSITY column by dividing vehicle count by road length
    print("Creating VEHICLE_INTENSITY column:", end=' ')

    vehicle_count = round(when(
        col("LINK_LENGTH_KM") != 0,
        col("MOTER_VEHICLAS_COUNT") / col("LINK_LENGTH_KM")
    ).otherwise(None), 2)
    
    create_intensity_df = df.withColumn("VEHICLE_INTENSITY", vehicle_count) 
    
    print('Success !!')
    print("****************************************************************************************")
    return create_intensity_df


In [0]:
def create_loadTime(df):
    # Add current timestamp as LOAD_TIME column
    print("Adding LOAD_TIME column to DataFrame:", end=' ')
    
    load_df = df.withColumn("LOAD_TIME", current_timestamp())
    
    print('Success !!')
    print("****************************************************************************************")
    return load_df


In [0]:
def write_gold_traffic_data(df, environment):
    # Write data to the gold_traffic table
    print('Starting to write the Gold Road Table:', end=' ')
    
    result_traffic.writeStream\
                 .format("delta")\
                 .option("checkpointLocation", checkpoint_path +"/goldrawtrafficload/Checkpt")\
                 .outputMode("append") \
                 .queryName("goldTrafficWriteStream") \
                 .trigger(availableNow=True) \
                 .toTable(f"{environment}_catalog.gold.gold_traffic")

    print('Success !!')
    print("****************************************************************************************")


In [0]:
def write_gold_roads_data(df, environment ):
    # Write data to the gold_roads table
    print('Starting to write the Gold Road Table:', end=' ')
    
    result_road.writeStream\
               .format("delta")\
               .option("checkpointLocation", checkpoint_path +"/goldrawroadload/Checkpt")\
               .outputMode("append")\
               .queryName("goldRoadWriteStream")\
               .trigger(availableNow=True)\
               .toTable(f"{environment}_catalog.gold.gold_roads")
    
    print('Success !!')
    print("****************************************************************************************")


In [0]:
# Read traffic data from silver layer
read_df = read_silver_traffic_data(env)

# Read road data from silver layer
roads_df = read_silver_roads_data(env)

# Create vehicle intensity column in the traffic data
intensity_df = create_vehicle_intensity(read_df)

# Add load time column to the traffic data
result_traffic = create_loadTime(intensity_df)

# Add load time column to the road data
result_road = create_loadTime(roads_df)

# Write traffic data to gold layer
write_traffic = write_gold_traffic_data(result_traffic, env)

# Write road data to gold layer
write_road = write_gold_roads_data(result_road, env)


In [0]:
display(result_traffic)

In [0]:
%sql
SELECT COUNT(*) as total_row  FROM `dev_catalog`.`gold`.`gold_traffic`
UNION ALL SELECT COUNT(*)  FROM `dev_catalog`.`gold`.`gold_roads`